## Load in Example file 

In [6]:
file_path = 'marijuana.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

print(len(file_content))


190425


## Connecting to Pinecone

In [1]:
%pip install pinecone-client
%pip install python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Load API key from .env


from dotenv import load_dotenv
import os

## Error Handling for API key retreival
try: 
            
    load_dotenv()

    PC_KEY = os.getenv('PINECONE_API_KEY')
    print(PC_KEY)

    if not PC_KEY:
        raise ValueError("PINECONE_API_KEY not found in .env file")


except Exception as e:
    print(f"Error: {e}")



623494db-40e1-44ee-9890-26f24e1dd55b


In [4]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PC_KEY)


## Create Index 

In [8]:
print(f"Indexes: {pc.list_indexes().names()}")
print(f"Collections: {pc.list_collections().names()}")

Indexes: []
Collections: []


In [9]:
## This POC currently only uses 1 index. As we scale up our data and decide how we can partition different legal documents, we can scale horizontally. 
## In this demo, there are only a handful of documents, enabling efficiency with a single index


index_name = "idx-one"

## Embedding model is [BERT large model (uncased)], which outputs vectors of [1024] dimensions
## Cosine similarity so search is not skewed by magnitude

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


In [10]:
print(f"Indexes: {pc.list_indexes().names()}") # 'idx-one' added successfully
print(f"Collections: {pc.list_collections().names()}")

Indexes: ['idx-one']
Collections: []


## Converting Text to Embeddings

In [11]:
%pip install transformers torch 

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     ---------------------------------------- 43.8/43.8 kB ? eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/9.1 MB 9.4 MB/s eta 0:00:01
   ---- ----------------------------------- 1.0/9.1 MB 10.5 MB/s eta 0:00:01
   ------- -------------------------------- 1.6/9.1 MB 11.5 MB/s eta 0:00:01
   --------- ------------------------------ 2.0/9.1 MB 11.8 MB/s eta 0:00:01
   ------------ -------------------------

In [12]:
from transformers import BertTokenizer, BertModel
import torch

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

C:\Users\prath\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prath\.cache\huggingface\hub\models--bert-large-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\prath\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:1132: Fu